In [1]:
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

In [21]:
apikey = os.environ.get('APIKEY_FREE')

In [22]:
# openai.api_key = apikey
# client = OpenAI(api_key=apikey)
client = OpenAI(
    api_key=apikey,
)

In [23]:
# engines = OpenAI.Engine
# engines

In [100]:
prompt = "Spinach Feta Quiche approx. 300g; small green salad with vinegar oil dressing; Potato soup 100ml, quark with pineapple approx. 100g"
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0613", 
    messages=[
    {"role": "system", "content": "You are a dietitian. Categorize the information you receive, using a table format. The table should contain: food, portion size, units. Use one word or one number only"},
    {"role": "user", "content": prompt}
],
    functions=[
    {
        "name": "get_food_breakdown",
        "description": "Return the keywords in the prompt which satisfy the criteria",
        "parameters": {
            "type": "object",
            "properties": {
                "food": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "a list of foods the prompt has"
                },
                "food_start_idx": {
                    "type": "array",
                    "items": {
                        "type": "number"
                    },
                    "description": "start index for the corresponding food"
                },
                "food_end_idx": {
                    "type": "array",
                    "items": {
                        "type": "number"
                    },
                    "description": "end index for the corresponding food"
                },
                "quantity": {
                    "type": "array",
                    "items": {
                        "type": "number"
                    },
                    "description": "list of numbers, corresponding to the above foods"
                },
                "qty_start_idx": {
                    "type": "array",
                    "items": {
                        "type": "number"
                    },
                    "description": "start index for the corresponding quantity"
                },
                "qty_end_idx": {
                    "type": "array",
                    "items": {
                        "type": "number"
                    },
                    "description": "end index for the corresponding quantity"
                },
                "units": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "letters or words following and corresponding to the numbers from above"
                },
                "unit_start_idx": {
                    "type": "array",
                    "items": {
                        "type": "number"
                    },
                    "description": "start index for the corresponding units"
                },
                "unit_end_idx": {
                    "type": "array",
                    "items": {
                        "type": "number"
                    },
                    "description": "end index for the corresponding units"
                }
            },
            "required": ["food","food_start_idx", 'food_end_idx', "quantity", "qty_start_idx", "qty_end_idx", "units", "unit_start_idx", "unit_end_idx"]
        }
    }
],
function_call="auto",
max_tokens=300

)


In [101]:
completion.choices[0].message.function_call.arguments

'{\n  "food": ["Spinach Feta Quiche", "small green salad", "Potato soup", "quark with pineapple"],\n  "food_start_idx": [0, 23, 43, 57],\n  "food_end_idx": [18, 38, 55, 77],\n  "quantity": [300, "", 100, 100],\n  "qty_start_idx": [23, -1, 54, 73],\n  "qty_end_idx": [26, -1, 57, 76],\n  "units": ["g", "", "ml", "g"],\n  "unit_start_idx": [27, -1, 58, 78],\n  "unit_end_idx": [28, -1, 60, 79]\n}'

In [102]:
# args = json.loads(completion.choices[0].message.function_call.arguments)
args = json.loads(completion.choices[0].message.function_call.arguments)

In [103]:
pd.DataFrame.from_dict(args, orient='columns')

,food,food_start_idx,food_end_idx,quantity,qty_start_idx,qty_end_idx,units,unit_start_idx,unit_end_idx
0,Spinach Feta Quiche,0,18,300,23,26,g,27,28
1,small green salad,23,38,,-1,-1,,-1,-1
2,Potato soup,43,55,100,54,57,ml,58,60
3,quark with pineapple,57,77,100,73,76,g,78,79
